In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.profiler

from torch.utils.tensorboard import SummaryWriter
from dataset import MyDataset
from dataloader import PatchDataLoader
from torchvision import transforms
from PIL import Image
from model import CNN
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

Image.MAX_IMAGE_PIXELS = 933120000

In [2]:
torch.cuda.empty_cache()
save_log_dir = "deneme/1296"
writer = SummaryWriter(log_dir=save_log_dir)

In [3]:
def save_checkpoint(state, checkpoint, filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)

In [4]:
def draw_graph(loss, psnr, ssim, epoch):
    
    writer.add_scalar("Train Loss", loss, epoch)
    writer.add_scalar("PSNR/Epoch", psnr, epoch)
    writer.add_scalar("SSIM/Epoch", ssim, epoch)
    
    del psnr, loss

In [5]:
def plot_feature(image, model):
    no_of_layers= 0
    layers = []
    weights = []
    outputs = []
    processed = []
    names = []
    type_list = [nn.Conv2d, nn.AvgPool2d, nn.ConvTranspose2d, nn.Upsample]
    model_children=list(model.children())

    for child in model_children:
        if type(child) in type_list:
            no_of_layers+=1
            layers.append(child)
            if type(child)==nn.Conv2d:
                weights.append(child.weight)

    for layer in layers[0:]:
        image = layer(image)
        outputs.append(image)

    for feature_map in outputs:
        feature_map = feature_map.squeeze(0)
        gray_scale = torch.sum(feature_map,0)
        gray_scale = gray_scale / feature_map.shape[0]
        #for gray_scale in feature_map:
        processed.append(gray_scale.detach().cpu().numpy())
        names.append(str(gray_scale.shape))
        
    fig = plt.figure(figsize=(20, 700))
    for i in range(len(processed)):
        a = fig.add_subplot(1000, 10, i+1)
        imgplot = plt.imshow(processed[i], cmap="gray")
        a.axis("on")
        a.set_title(names[i], fontsize=8)
    plt.savefig(str(f'{save_log_dir}/feature_maps.jpg'), bbox_inches='tight')

    return fig

In [6]:
transform =transforms.Compose([transforms.ToTensor()])
dataset = MyDataset("image")
loader = PatchDataLoader(dataset=dataset, transform=transform, kernel_size=64, stride=64, batch_size=80)

In [7]:
dataiter = iter(loader)
images = next(dataiter)
img_grid = torchvision.utils.make_grid(images)
writer.add_image('Example input image.', img_grid)
del img_grid

In [8]:
model = CNN()
model.to("cuda")
model.train()
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
writer.add_graph(model, images)

In [9]:
epochs = 1000
summary = { 
        "BSIZE": loader.batch_size,
        "EPOCH": epochs, 
        "LFUNC": criterion, 
        "OPTIM": optimizer,
        "SIZE" : loader.size,
        "STRIDE": loader.stride,
        "BOTTLENECK": "NONE",
        "RELU": model.encoder[1],
        "DOWNSAMPLE": model.encoder[2],
        "UPSAMPLE": model.decoder[2]
        }

writer.add_text("Summary", str(summary))

In [10]:
""" with torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=10, warmup=10, active=2, repeat=0),
        on_trace_ready=torch.profiler.tensorboard_trace_handler(save_log_dir),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof: """
    
for epoch in range(epochs):
    for batch_idx, inputs in enumerate(loader,0):
        if inputs == None:
            break
        
        input = inputs.to("cuda")
        output = model(input)
        optimizer.zero_grad()
        loss = criterion(output, input)
        loss.backward()
        optimizer.step()
        
        if batch_idx == 0:            
            img_grid = torchvision.utils.make_grid(output)
            writer.add_image('First batch output image.', img_grid, global_step=epoch)
              
            
    save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }, save_log_dir)

    psnr_val = psnr(input[0].transpose(0,2).detach().cpu().numpy(), output[0].transpose(0,2).detach().cpu().numpy()) #psnr(input[0], output[0], max_val=1.)
    ssim_val = ssim(input[0].transpose(0,2).detach().cpu().numpy(), output[0].transpose(0,2).detach().cpu().numpy(), channel_axis=2)
    print(f"Epoch: {epoch}/{epochs}, Loss: {loss}, PSNR: {psnr_val}, SSIM: {ssim_val}")
    draw_graph(loss=loss, psnr=psnr_val, ssim=ssim_val, epoch=epoch)

Epoch: 0/1000, Loss: 0.17979475855827332, PSNR: 13.155492026012439, SSIM: 0.45580044388771057
Epoch: 1/1000, Loss: 0.12790969014167786, PSNR: 17.223132659611373, SSIM: 0.6109716296195984
Epoch: 2/1000, Loss: 0.11429136246442795, PSNR: 19.687307048024685, SSIM: 0.6853308081626892
Epoch: 3/1000, Loss: 0.10806117951869965, PSNR: 20.77894461375845, SSIM: 0.7267032265663147
Epoch: 4/1000, Loss: 0.10288406163454056, PSNR: 21.41796209697905, SSIM: 0.7551751136779785
Epoch: 5/1000, Loss: 0.09556739032268524, PSNR: 21.122199437047197, SSIM: 0.7422000765800476
Epoch: 6/1000, Loss: 0.09154292196035385, PSNR: 21.121725314468485, SSIM: 0.7500969767570496
Epoch: 7/1000, Loss: 0.08882889151573181, PSNR: 21.489189774155754, SSIM: 0.7683046460151672
Epoch: 8/1000, Loss: 0.08500010520219803, PSNR: 21.598458634383654, SSIM: 0.7732951045036316
Epoch: 9/1000, Loss: 0.08438432961702347, PSNR: 21.880389744877114, SSIM: 0.7847268581390381
Epoch: 10/1000, Loss: 0.08232075721025467, PSNR: 22.09142741097306, SSI